# Keras Tuner Bars

In [1]:
import numpy as np
import pandas as pd
import sklearn.preprocessing
import time
import sys
import copy

sys.path.append("..")
from helpers import filename_for

Welcome to JupyROOT 6.16/00


In [2]:
import tensorflow
from tensorflow import keras
import kerastuner

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)
print("keras-tuner", kerastuner.__version__)

tensorflow 2.1.0
keras 2.2.4-tf
keras-tuner 1.0.1


2020-08-21 16:17:29.734662: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/jmayer/Simulations/R3BRoot/build/lib:/data/jmayer/Simulations/2019-09-04/ucesb/hbook:/data/jmayer/Simulations/2019-09-04/FairRoot/lib:/data/jmayer/Simulations/2019-09-04/FairSoft/lib/root:/data/jmayer/Simulations/2019-09-04/FairSoft/lib
2020-08-21 16:17:29.735050: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/jmayer/Simulations/R3BRoot/build/lib:/data/jmayer/Simulations/2019-09-04/ucesb/hbook:/data/jmayer/Simulations/2019-09-04/FairRoot/lib:/data/jmayer/Simulations/2019-09-04/FairSoft/lib/root:/data/jmayer/Simulations/2019-09-04/FairSoft/lib
2020-08-21 1

In [3]:
files = [filename_for(15, 30, 600, 500, n, "inclxx", s, "bars.parquet") for n in [1, 2, 3, 4] for s in range(20)]
dfs = [pd.read_parquet(file) for file in files]
data = pd.concat(dfs, ignore_index=True).sample(frac=1)
data.loc[data["nHits"] == 0, "nPN"] = 0
display(data)

,nPN,nPP,nPH,nHits,nClus,Edep,0,1,2,3,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
752998,4,4,4,52,25,988,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402997,3,3,3,32,15,660,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
642698,4,4,4,66,35,886,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
520488,3,3,3,60,23,893,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
439930,3,3,3,37,12,616,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193456,1,1,1,19,3,327,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69487,1,1,1,12,6,228,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236017,2,2,2,22,12,369,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48513,1,1,1,19,15,192,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
cols_tri = ["nHits", "nClus", "Edep"]
cols_e = [str(i) for i in range(0, 30 * 100 * 2, 2)]
cols_t = [str(i + 1) for i in range(0, 30 * 100 * 2, 2)]

scaler = sklearn.preprocessing.MaxAbsScaler()
s_tri = copy.copy(scaler).fit(data[cols_tri])
s_e = copy.copy(scaler).fit(data[cols_e].values.reshape(-1, 1))
s_t = copy.copy(scaler).fit(data[cols_t].values.reshape(-1, 1))

data_scaled = np.concatenate(
    (
        s_tri.transform(data[cols_tri]),
        s_e.transform(data[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
        s_t.transform(data[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
    ),
    axis=1,
)
display(data_scaled)

array([[0.34437086, 0.47169811, 0.34293648, ..., 0.        , 0.        ,
        0.        ],
       [0.21192053, 0.28301887, 0.22908712, ..., 0.        , 0.        ,
        0.        ],
       [0.43708609, 0.66037736, 0.30753211, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.14569536, 0.22641509, 0.12808053, ..., 0.        , 0.        ,
        0.        ],
       [0.12582781, 0.28301887, 0.06664353, ..., 0.        , 0.        ,
        0.        ],
       [0.2384106 , 0.24528302, 0.1832697 , ..., 0.        , 0.        ,
        0.        ]])

In [5]:
label = "nPN"
max_neutrons = 4
num_classes = max_neutrons + 1

ys = keras.utils.to_categorical(data[[label]].values.ravel())
Xs = data_scaled

In [6]:
class Model(kerastuner.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.models.Sequential()
        model.add(keras.layers.Dense(units=hp.Int(f"relu1nodes", min_value=1000, max_value=10000, step=3000), activation="relu",))
        model.add(keras.layers.Dense(units=hp.Int(f"relu2nodes", min_value=500, max_value=3000, step=200), activation="relu",))
        model.add(keras.layers.Dense(units=self.num_classes, activation="softmax"))

        optimizer = hp.Choice("optimizer", ["adam", "adagrad", "adadelta", "SGD"])
        loss = keras.losses.CategoricalCrossentropy()

        model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

        return model

In [7]:
model = Model(num_classes)

tuner = kerastuner.tuners.BayesianOptimization(
#tuner = kerastuner.tuners.RandomSearch(
    model, objective="val_accuracy", max_trials=20, executions_per_trial=1, directory="kerastuner", project_name="V1", overwrite=True
)
tuner.search_space_summary()

In [ ]:
tuner.search(
    Xs, ys, verbose=2, validation_split=0.1
)

Train on 720000 samples, validate on 80000 samples
720000/720000 - 1665s - loss: 0.6293 - accuracy: 0.7252 - val_loss: 0.6087 - val_accuracy: 0.7337


Train on 720000 samples, validate on 80000 samples


In [ ]:
tuner.results_summary()